In [13]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report


In [128]:

# Load the dfset
df = pd.read_csv("/home/vikasnr/codebase/crsl/tuesday/modified_file.csv")  # Replace with actual filename
# df.head(2)



In [129]:
# df.data_type.value_counts()

In [130]:
from sklearn.preprocessing import MultiLabelBinarizer
import pandas as pd
from ast import literal_eval


# Drop the original column if not needed
df = df.drop(columns=["data_type"])


# Display the transformed DataFrame
df.head(1)

,PID,prompt_length,complexity,module,num_tokens,num_sentences,lexical_diversity,readability,noun_ratio,verb_ratio,...,num_named_entities,contains_table,contains_list,redundancy_score,compression_ratio,contains_numbers,contains_chain_of_thought,contains_output_constraints,is_multi_turn,LLM
0,0,60,Simple,document_extraction,11,1,0.909091,8.4,0.272727,0.181818,...,1,False,False,1,5.454545,False,False,False,False,GPT


In [131]:

# df.data_type.value_counts()

# Text                                             183
# ['Text', 'Tables']                                91
# Numerical Data                                    83
# ['Text', 'Graphs & Charts']                       70
# ['Text']                                          24
# ['Text', 'Numerical Data']                        16
# ['Numerical Data']                                 9
# ['Numerical Data', 'Tables']                       9
# ['Dates & Timestamps']                             5
# ['Equations & Formulas']                           4
# ['Text', 'Numerical Data', 'Graphs & Charts']      3
# ['Text', 'Tables', 'Graphs & Charts']              3
# ['Graphs & Charts', 'Numerical Data']              3
# ['Graphs & Charts']                                1
# ['Numerical Data', 'Text']                         1
# ['Tables']                                         1
# ['Text', 'Numerical Data', 'Tables']               1
# graph                                               1
# Name: count, dtype: int64

# I want columns to be like this:
# Text, Numerical Data, Tables, Graphs & Charts, Dates & Timestamps, Equations & Formulas
# 1, 0, 0, 0, 0, 0

#### drop PIP

In [132]:
# Drop PID column as it's just an identifier
df = df.drop(columns=["PID"])

df.head()

,prompt_length,complexity,module,num_tokens,num_sentences,lexical_diversity,readability,noun_ratio,verb_ratio,adjective_ratio,num_named_entities,contains_table,contains_list,redundancy_score,compression_ratio,contains_numbers,contains_chain_of_thought,contains_output_constraints,is_multi_turn,LLM
0,60,Simple,document_extraction,11,1,0.909091,8.4,0.272727,0.181818,0.000000,1,False,False,1,5.454545,False,False,False,False,GPT
1,55,Simple,document_extraction,10,1,1.000000,9.2,0.200000,0.200000,0.000000,0,False,False,0,5.500000,False,False,False,False,GPT
2,95,Simple,document_extraction,17,1,0.941176,10.3,0.235294,0.176471,0.000000,0,False,False,1,5.588235,False,False,False,False,GPT
3,97,Simple,document_extraction,19,1,0.842105,9.5,0.315789,0.105263,0.052632,0,False,False,2,5.105263,False,False,False,False,GPT
4,71,Simple,document_extraction,13,1,1.000000,9.9,0.153846,0.153846,0.076923,1,False,False,1,5.461538,False,False,False,False,GPT


##### Encode the target variable (LLM)


In [133]:
le = LabelEncoder()
df["LLM"] = le.fit_transform(df["LLM"])
df.head()


,prompt_length,complexity,module,num_tokens,num_sentences,lexical_diversity,readability,noun_ratio,verb_ratio,adjective_ratio,num_named_entities,contains_table,contains_list,redundancy_score,compression_ratio,contains_numbers,contains_chain_of_thought,contains_output_constraints,is_multi_turn,LLM
0,60,Simple,document_extraction,11,1,0.909091,8.4,0.272727,0.181818,0.000000,1,False,False,1,5.454545,False,False,False,False,1
1,55,Simple,document_extraction,10,1,1.000000,9.2,0.200000,0.200000,0.000000,0,False,False,0,5.500000,False,False,False,False,1
2,95,Simple,document_extraction,17,1,0.941176,10.3,0.235294,0.176471,0.000000,0,False,False,1,5.588235,False,False,False,False,1
3,97,Simple,document_extraction,19,1,0.842105,9.5,0.315789,0.105263,0.052632,0,False,False,2,5.105263,False,False,False,False,1
4,71,Simple,document_extraction,13,1,1.000000,9.9,0.153846,0.153846,0.076923,1,False,False,1,5.461538,False,False,False,False,1


#### Separate features and target



In [134]:
y = df["LLM"]
X = df.drop(columns=["LLM"])

print(X.columns)

Index(['prompt_length', 'complexity', 'module', 'num_tokens', 'num_sentences',
       'lexical_diversity', 'readability', 'noun_ratio', 'verb_ratio',
       'adjective_ratio', 'num_named_entities', 'contains_table',
       'contains_list', 'redundancy_score', 'compression_ratio',
       'contains_numbers', 'contains_chain_of_thought',
       'contains_output_constraints', 'is_multi_turn'],
      dtype='object')


##### one-hot encoding of categorical features

In [135]:
# get categorical columns
cat_columns = X.select_dtypes(['object']).columns
cat_columns

Index(['complexity', 'module'], dtype='object')

In [136]:
categorical_cols = ["complexity", "module"]  # Exclude "LLM"
ohe = OneHotEncoder(sparse_output=False, handle_unknown="ignore")

# Fit and transform only the selected categorical columns
categorical_encoded = ohe.fit_transform(X[categorical_cols])

# Convert to DataFrame
categorical_df = pd.DataFrame(categorical_encoded, columns=ohe.get_feature_names_out(categorical_cols))

categorical_df.head()
X = pd.concat([X, categorical_df], axis=1)
X = X.drop(columns=categorical_cols)

X.head()


,prompt_length,num_tokens,num_sentences,lexical_diversity,readability,noun_ratio,verb_ratio,adjective_ratio,num_named_entities,contains_table,...,contains_output_constraints,is_multi_turn,complexity_Complex,complexity_Medium,complexity_Simple,module_document_extraction,module_interactive_qa_chatbot,module_multi_document_summarization,module_quantitative_analysis,module_report_generation
0,60,11,1,0.909091,8.4,0.272727,0.181818,0.000000,1,False,...,False,False,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0
1,55,10,1,1.000000,9.2,0.200000,0.200000,0.000000,0,False,...,False,False,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0
2,95,17,1,0.941176,10.3,0.235294,0.176471,0.000000,0,False,...,False,False,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0
3,97,19,1,0.842105,9.5,0.315789,0.105263,0.052632,0,False,...,False,False,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0
4,71,13,1,1.000000,9.9,0.153846,0.153846,0.076923,1,False,...,False,False,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0


In [137]:
type(ohe)

sklearn.preprocessing._encoders.OneHotEncoder

In [138]:
type(categorical_encoded)

numpy.ndarray

In [139]:
df.columns

Index(['prompt_length', 'complexity', 'module', 'num_tokens', 'num_sentences',
       'lexical_diversity', 'readability', 'noun_ratio', 'verb_ratio',
       'adjective_ratio', 'num_named_entities', 'contains_table',
       'contains_list', 'redundancy_score', 'compression_ratio',
       'contains_numbers', 'contains_chain_of_thought',
       'contains_output_constraints', 'is_multi_turn', 'LLM'],
      dtype='object')

In [140]:
df.head(2)

,prompt_length,complexity,module,num_tokens,num_sentences,lexical_diversity,readability,noun_ratio,verb_ratio,adjective_ratio,num_named_entities,contains_table,contains_list,redundancy_score,compression_ratio,contains_numbers,contains_chain_of_thought,contains_output_constraints,is_multi_turn,LLM
0,60,Simple,document_extraction,11,1,0.909091,8.4,0.272727,0.181818,0.0,1,False,False,1,5.454545,False,False,False,False,1
1,55,Simple,document_extraction,10,1,1.000000,9.2,0.200000,0.200000,0.0,0,False,False,0,5.500000,False,False,False,False,1


#### Test Train split

In [141]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [156]:
subset = X.head(4)
subset.to_csv("X_train_subset.csv", index=False)


In [142]:
X_train.head()

,prompt_length,num_tokens,num_sentences,lexical_diversity,readability,noun_ratio,verb_ratio,adjective_ratio,num_named_entities,contains_table,...,contains_output_constraints,is_multi_turn,complexity_Complex,complexity_Medium,complexity_Simple,module_document_extraction,module_interactive_qa_chatbot,module_multi_document_summarization,module_quantitative_analysis,module_report_generation
475,90,14,1,1.000000,13.9,0.285714,0.142857,0.214286,0,False,...,False,False,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
499,85,14,1,1.000000,16.2,0.214286,0.142857,0.142857,1,False,...,False,False,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
410,64,12,1,0.916667,7.6,0.250000,0.083333,0.083333,0,False,...,False,False,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
273,52,9,1,1.000000,8.8,0.444444,0.111111,0.111111,1,False,...,False,False,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
291,69,13,1,0.846154,9.2,0.230769,0.153846,0.000000,0,False,...,False,False,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0


#### Feature scaling

In [143]:

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)



#### Train the classifier


In [144]:
!pip install xgboost

In [145]:
from xgboost import XGBClassifier

model = XGBClassifier()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print(accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))


0.7352941176470589
              precision    recall  f1-score   support

           0       0.67      0.40      0.50        15
           1       0.83      0.68      0.75        22
           2       0.73      0.75      0.74        32
           3       0.71      0.91      0.80        33

    accuracy                           0.74       102
   macro avg       0.74      0.69      0.70       102
weighted avg       0.74      0.74      0.73       102



In [146]:
import joblib
#save model
joblib.dump(model, "xgboost_v2.pkl") 

# #load saved model
# xgb = joblib.load("xgboost.pkl")
# xgb.predict(X_test)

['xgboost_v2.pkl']

In [148]:
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)


RandomForestClassifier(random_state=42)

##### Predict on test data


In [65]:

y_pred = clf.predict(X_test)
y_pred

array([2, 2, 2, 1, 3, 3, 2, 1, 2, 1, 2, 3, 2, 2, 2, 1, 3, 3, 2, 2, 0, 3,
       2, 1, 1, 3, 3, 1, 0, 3, 2, 1, 3, 2, 1, 2, 2, 2, 3, 3, 3, 1, 1, 3,
       3, 3, 2, 3, 1, 3, 3, 3, 3, 3, 3, 2, 1, 1, 1, 2, 3, 1, 0, 3, 0, 1,
       2, 2, 3, 2, 0, 0, 3, 3, 1, 3, 0, 2, 0, 1, 0, 3, 0, 1, 2, 2, 0, 0,
       2, 0, 1, 3, 1, 2, 3, 3, 0, 2, 2, 2, 1, 0])

#### Model evaluation

In [149]:
# Evaluate the model
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score

# Convert integer labels back to class names
y_test_labels = le.inverse_transform(y_test)  # Convert encoded labels to original class names
y_pred_labels = le.inverse_transform(y_pred)  # Convert predicted labels to original class names

print(classification_report(y_test_labels, y_pred_labels, target_names=le.classes_))
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")
precision_score = precision_score(y_test, y_pred, average='weighted')
print(f"Precision: {precision_score:.2f}")
recall_score = recall_score(y_test, y_pred, average='weighted')
print(f"Recall: {recall_score:.2f}")
f1_score = f1_score(y_test, y_pred, average='weighted')
print(f"F1 Score: {f1_score:.2f}")

              precision    recall  f1-score   support

      Claude       0.67      0.40      0.50        15
         GPT       0.83      0.68      0.75        22
     Mistral       0.73      0.75      0.74        32
        llma       0.71      0.91      0.80        33

    accuracy                           0.74       102
   macro avg       0.74      0.69      0.70       102
weighted avg       0.74      0.74      0.73       102

Accuracy: 0.74
Precision: 0.74
Recall: 0.74
F1 Score: 0.73


In [150]:
import joblib

# Save the MultiLabelBinarizer model
joblib.dump(clf, "random_forest_model.pkl")

# Save the LabelEncoder
joblib.dump(le, "random_forest_label_encoder.pkl")

# Save the StandardScaler
joblib.dump(scaler, "random_forest_scaler.pkl")

# Save the OneHotEncoder
joblib.dump(ohe, "random_forest_onehot_encoder.pkl")


['random_forest_onehot_encoder.pkl']

##### Inference

##### Extract features

In [68]:
import spacy
import textstat
import re
from collections import Counter

# Load spaCy NLP model
nlp = spacy.load("en_core_web_sm")

def extract_prompt_features(prompt):
    doc = nlp(prompt)
    
    #  Linguistic Features
    num_tokens = len(doc)
    num_sentences = len(list(doc.sents))
    unique_words = len(set([token.text.lower() for token in doc]))
    lexical_diversity = unique_words / num_tokens if num_tokens > 0 else 0
    readability = textstat.flesch_kincaid_grade(prompt)
    
    # Part-of-Speech (POS) Distribution
    pos_counts = Counter([token.pos_ for token in doc])
    noun_ratio = pos_counts.get("NOUN", 0) / num_tokens
    verb_ratio = pos_counts.get("VERB", 0) / num_tokens
    adjective_ratio = pos_counts.get("ADJ", 0) / num_tokens

    #  Structural Features
    num_named_entities = len(doc.ents)
    contains_table = bool(re.search(r"(\||\+\-+)|Table \d+", prompt))  # Detect tables
    contains_list = bool(re.search(r"(\d+\.)|(- )|(\* )", prompt))  # Detect bullet points or numbered lists

    # Content-Specific Features
    keyword_density = {word: prompt.lower().count(word) for word in ["summary", "analyze", "data", "report"]}
    redundancy_score = len(re.findall(r"(\b\w+\b).*\1", prompt))  # Count repeated words
    compression_ratio = len(prompt) / num_tokens if num_tokens > 0 else 0

    #  Task-Specific Complexity Features
    contains_numbers = bool(re.search(r"\d+", prompt))  # Presence of numerical data
    contains_chain_of_thought = bool(re.search(r"step-by-step|explain your reasoning", prompt.lower()))
    contains_output_constraints = bool(re.search(r"limit to|output in|return a json", prompt.lower()))
    is_multi_turn = "previous response" in prompt.lower() or "as mentioned before" in prompt.lower()

    return {
        "num_tokens": num_tokens,
        "num_sentences": num_sentences,
        "lexical_diversity": lexical_diversity,
        "readability": readability,
        "noun_ratio": noun_ratio,
        "verb_ratio": verb_ratio,
        "adjective_ratio": adjective_ratio,
        "num_named_entities": num_named_entities,
        "contains_table": contains_table,
        "contains_list": contains_list,
        "redundancy_score": redundancy_score,
        "compression_ratio": compression_ratio,
        "contains_numbers": contains_numbers,
        "contains_chain_of_thought": contains_chain_of_thought,
        "contains_output_constraints": contains_output_constraints,
        "is_multi_turn": is_multi_turn
    }




##### Load model

In [151]:

import joblib
# load the model and use it to make new predictions
model = joblib.load("random_forest_model.pkl")
scaler = joblib.load("random_forest_scaler.pkl")
ohe = joblib.load("random_forest_onehot_encoder.pkl")
le = joblib.load("random_forest_label_encoder.pkl")


def classify_complexity(prompt, num_tokens=25, readability=70, num_sentences=1, lexical_diversity=2, num_named_entities=3):
    if num_tokens <= 10 and readability >= 60 and num_sentences == 1:
        return "Simple"
    elif 10 < num_tokens <= 20 or (40 <= readability < 60) or (num_named_entities <= 4):
        return "Medium"
    else:
        return "Complex"


print()  # Output: Medium


In [ ]:
# write a prompt for summarizing a text


In [164]:
import pandas as pd
# Convert input into DataFrame
# new_df = pd.DataFrame([new_input])
# Example Prompt
prompt = """
You are an AI assistant. Summarize the research paper and provide key insights. 
Ensure the response is formatted as JSON and limited to 200 words. 
Step-by-step analysis is preferred. Example: "In this study, the authors examined..."
"""
prompt = "What was the main finding of the report?"
# Extract Features
features = {}
X_train_subset = pd.read_csv("X_train_subset.csv")

features = extract_prompt_features(prompt)
complexity = classify_complexity(prompt)
features.update({"complexity": complexity, "module": "document_extraction","prompt_length": len(prompt)})
print(features)

df = pd.DataFrame([features],index=[0])

# Transform the input using the saved encoders


categorical_cols = ["complexity", "module"]
categorical_encoded = ohe.transform(df[categorical_cols])
categorical_df = pd.DataFrame(categorical_encoded, columns=ohe.get_feature_names_out())



df = pd.concat([df, categorical_df], axis=1)
new_df = df.drop(columns=categorical_cols)

new_df.head()
new_df = new_df[X_train_subset.columns]
new_df = scaler.transform(new_df)

# # # Make a prediction
prediction = model.predict(new_df)
predicted_class = le.inverse_transform(prediction)[0]
print(f"Predicted class: {predicted_class}")


# Index(['prompt_length', 'complexity', 'module', 'num_tokens', 'num_sentences',
#        'lexical_diversity', 'readability', 'noun_ratio', 'verb_ratio',
#        'adjective_ratio', 'num_named_entities', 'contains_table',
#        'contains_list', 'redundancy_score', 'compression_ratio',
#        'contains_numbers', 'contains_chain_of_thought',
#        'contains_output_constraints', 'is_multi_turn', 'LLM'],
#       dtype='object')



{'num_tokens': 9, 'num_sentences': 1, 'lexical_diversity': 0.8888888888888888, 'readability': 2.9, 'noun_ratio': 0.2222222222222222, 'verb_ratio': 0.0, 'adjective_ratio': 0.1111111111111111, 'num_named_entities': 0, 'contains_table': False, 'contains_list': False, 'redundancy_score': 1, 'compression_ratio': 4.444444444444445, 'contains_numbers': False, 'contains_chain_of_thought': False, 'contains_output_constraints': False, 'is_multi_turn': False, 'complexity': 'Medium', 'module': 'document_extraction', 'prompt_length': 40}
Predicted class: GPT


In [162]:
prob_scores = model.predict_proba(new_df)
print(prob_scores)
# Get predicted class and confidence score
predicted_class = model.predict(new_df)[0]
confidence_score = np.max(prob_scores) * 100  # Convert to percentage

print(f"Predicted Class: {predicted_class}")
print(f"Confidence Score: {confidence_score:.2f}%")

[[0.30594048 0.61805952 0.04125    0.03475   ]]
Predicted Class: 1
Confidence Score: 61.81%


In [163]:
model.predict(X_test)

array([2, 2, 3, 1, 3, 3, 2, 3, 2, 1, 2, 3, 3, 2, 2, 3, 3, 3, 2, 2, 2, 3,
       3, 1, 1, 3, 2, 2, 0, 3, 2, 1, 3, 3, 2, 2, 1, 2, 3, 3, 3, 1, 1, 2,
       3, 2, 0, 3, 0, 3, 3, 3, 3, 3, 3, 2, 1, 2, 1, 2, 3, 1, 2, 3, 1, 2,
       2, 2, 3, 0, 3, 3, 3, 3, 3, 3, 0, 2, 1, 1, 3, 3, 0, 1, 2, 2, 0, 3,
       2, 3, 1, 3, 1, 2, 3, 3, 0, 2, 2, 2, 1, 0])